In [1]:
import torch
import torch_geometric as pyg
import ugs_sampler
from torch_geometric.loader import DataLoader
# import matplotlib.pyplot as plt
# import networkx as nx
from ss_gnn import SubgraphGINEncoder

In [2]:
pepfunc_data = pyg.datasets.LRGBDataset("../data", "Peptides-func")
loader = DataLoader(pepfunc_data,batch_size=64,shuffle = True)

In [3]:
"""
def sampler_fallback(edge_index,ptr,m,k):
    try:
        return ugs_sampler.sample_batch(edge_index,ptr,m,k)
    except Exception as e:
        ptr = ptr.tolist()
        sizes = [ ptr[i+1]-ptr[i] for i in range(len(ptr)-1)]
        print("Smallest graph size: ",min(sizes))
        print("\t",sizes)


for batch in loader:
    try:
        a = ugs_sampler.sample_batch(batch.edge_index, batch.ptr, 1000,10)
    except:
        sizes = torch.bincount(batch.batch)
        min_size = sizes.min().item()
        print("min size:", min_size)


def func(d):
    try:
        a = ugs_sampler.create_preproc(d.edge_index, d.x.size(0), 10)
        b = ugs_sampler.sample(a, 100, 10)
        ugs_sampler.destroy_preproc(a)
        return b
    except Exception as e:
        print(d.x.size(0))
        G = pyg.utils.to_networkx(d)
        nx.draw(G)
        plt.show()

for d in pepfunc_data:
    func(d)
"""

'\ndef sampler_fallback(edge_index,ptr,m,k):\n    try:\n        return ugs_sampler.sample_batch(edge_index,ptr,m,k)\n    except Exception as e:\n        ptr = ptr.tolist()\n        sizes = [ ptr[i+1]-ptr[i] for i in range(len(ptr)-1)]\n        print("Smallest graph size: ",min(sizes))\n        print("\t",sizes)\n\n\nfor batch in loader:\n    try:\n        a = ugs_sampler.sample_batch(batch.edge_index, batch.ptr, 1000,10)\n    except:\n        sizes = torch.bincount(batch.batch)\n        min_size = sizes.min().item()\n        print("min size:", min_size)\n\n\ndef func(d):\n    try:\n        a = ugs_sampler.create_preproc(d.edge_index, d.x.size(0), 10)\n        b = ugs_sampler.sample(a, 100, 10)\n        ugs_sampler.destroy_preproc(a)\n        return b\n    except Exception as e:\n        print(d.x.size(0))\n        G = pyg.utils.to_networkx(d)\n        nx.draw(G)\n        plt.show()\n\nfor d in pepfunc_data:\n    func(d)\n'

In [3]:
model = SubgraphGINEncoder(in_channels=9)
model.to("cuda")

SubgraphGINEncoder(
  (node_proj): Linear(in_features=9, out_features=64, bias=True)
  (gin_convs): ModuleList(
    (0-2): 3 x GINConv(nn=MLP(
      (mlp): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
    ))
  )
  (final_lin): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
  )
)

In [4]:
for batch in loader:
    node_s, edge_index_s, edge_ptr_s, graph_id = ugs_sampler.sample_batch(batch.edge_index,batch.ptr, 300, 20)
    x = batch.x.float().to("cuda")
    output = model(x, node_s, edge_index_s, edge_ptr_s, graph_id, 20)
    # print(output[0].shape)
    # print(node_s.shape)
